In [10]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from tifffile import memmap

from caImageAnalysis import BrukerFish, VolumeFish
from caImageAnalysis.caiman_process import caiman_mcorr
from caImageAnalysis.mesm import *
from caImageAnalysis.visualize import visualize_images, visualize_volumes

pd.options.display.max_colwidth = 120

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Change the exp_folder to the path of the experiment

In [17]:
# Change this path to the experiment folder
exp_folder = r'/Volumes/Rapid/DOI/Bruker/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104'
region = ''

In [18]:
# Initialize the fish. 
# If custom 2P, use VolumeFish. If Bruker 2P, use BrukerFish.

# fish = VolumeFish(exp_folder)
fish = BrukerFish(exp_folder, region=region)
fish.data_paths

/Users/minelarinel/caImageAnalysis/caImageAnalysis/bruker_2P/voltage_output.py:22: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 22 of the file /Users/minelarinel/caImageAnalysis/caImageAnalysis/bruker_2P/voltage_output.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  Bs_data = BeautifulSoup(xml)
/Users/minelarinel/mambaforge/envs/mescore/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that usi

{'frametimes': PosixPath('/Volumes/Rapid/DOI/Bruker/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/motor_pretectum-000_frametimes.h5'),
 'raw_image': PosixPath('/Volumes/Rapid/DOI/Bruker/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/motor_pretectum-000_ch2.tif'),
 'raw': PosixPath('/Volumes/Rapid/DOI/Bruker/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/motor_pretectum-000'),
 'anatomy': None,
 'references': PosixPath('/Volumes/Rapid/DOI/Bruker/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/motor_pretectum-000/References'),
 'voltage_output': PosixPath('/Volumes/Rapid/DOI/Bruker/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/motor_pretectum-000/motor_pretectum-000_Cycle00001_VoltageOutput_001.xml'),
 'log': PosixPath('/Volumes/Rapid/DOI/Bruker/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/motor_pretectum-000/motor_pretectum-000.xml'),
 'volumes': {'0': {'image': PosixPath('/Volumes/Rapid/DOI/Bruker/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/img_stack_0/image.tif'),
   'frametimes': PosixPath('

## IF VOLUMETRIC: Pick a plane to apply movement correction

In [5]:
iw = visualize_volumes(fish)
iw.show()

KeyError: 'volumes'

In [5]:
iw.plot.canvas.close()

In [19]:
# Change this to the plane you want to test motion correction parameters on
plane = 0

## Set parameters

In [5]:
# motion correction parameters
strides = (80, 80)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (30, 30)         # overlap between pathes (size of patch strides+overlaps)
max_shifts = (20, 20)          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 20     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

In [6]:
opts_dict = {'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid}

## Motion correct

In [7]:
if fish.volumetric:
    mcorr_img = caiman_mcorr(fish, plane=plane, **opts_dict)
else:
    mcorr_img = caiman_mcorr(fish, **opts_dict)

  0%|          | 0/1 [00:00<?, ?it/s]

Decode mmap filename /Users/minelarinel/Documents/Duke University/Naumann Lab/DOI/Brain imaging/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/motor_pretectum-000_ch2_els__d1_1024_d2_1024_d3_1_order_F_frames_2046.mmap


100%|██████████| 1/1 [01:00<00:00, 60.75s/it]


Decode mmap filename /Users/minelarinel/Documents/Duke University/Naumann Lab/DOI/Brain imaging/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/motor_pretectum-000_ch2_els__d1_1024_d2_1024_d3_1_order_F_frames_2046.mmap
/Users/minelarinel/Documents/Duke University/Naumann Lab/DOI/Brain imaging/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/memmap_d1_1024_d2_1024_d3_1_order_C_frames_2046.mmap
Decode mmap filename /Users/minelarinel/Documents/Duke University/Naumann Lab/DOI/Brain imaging/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/memmap_d1_1024_d2_1024_d3_1_order_C_frames_2046.mmap


In [ ]:
# Compare the results
if fish.volumetric:
    orig = memmap(fish.data_paths['volumes'][str(plane)]['image'])
else:
    orig = memmap(fish.data_paths['raw_image'])
caiman_iw = visualize_images([orig, mcorr_img], names=['original', 'mcorr'])
caiman_iw.show()

RFBOutputContext()

     4159376 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159378 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159379 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159381 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159381 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159382 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159384 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159386 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159388 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     41593

In [9]:
caiman_iw.plot.canvas.close()

## Optimize parameters with Mesmerize

### Motion correction parameters

In [20]:
# Set combination of values to try
if 'vagal' in fish.data_paths['raw'].name:
    strides = [100]  # start a new patch for pw-rigid motion correction every x pixels
    overlaps = [50]  # overlap between patches (size of patch strides+overlaps)
    max_shifts = [30]  # maximum allowed rigid shifts (in pixels)
    max_deviation_rigid = [10]  # maximum shifts deviation allowed for patch with respect to rigid shifts

else:  # for hindbrain and other brain regions
    strides = [80]
    overlaps = [30]
    max_shifts = [20]
    max_deviation_rigid = [20]

In [21]:
# Add the combination of parameters to a mes_df dataframe
img_path = fish.data_paths['volumes'][str(plane)]['image']
mes_df = add_mcorr(fish, [img_path], grid=True, 
                         strides=strides, 
                         overlaps=overlaps,
                         max_shifts=max_shifts,
                         max_deviation_rigid=max_deviation_rigid)

Batch does not exist. Creating batch.pickle


In [22]:
mes_df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,img_stack_0,img_stack_0/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (30, 30), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2024-03-29T18:46:26,None,None,None,ca450d2a-f550-4abd-99ba-771d2e4dd7dd


In [23]:
# See the unique parameters between rows with the same item_name
diffs = mes_df.caiman.get_params_diffs(algo="mcorr", item_name=mes_df.iloc[0]["item_name"])
diffs

/var/folders/48/8y2gqcvx5q5575kwkzngl6qw0000gn/T/ipykernel_14050/688498701.py:2: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  diffs = mes_df.caiman.get_params_diffs(algo="mcorr", item_name=mes_df.iloc[0]["item_name"])


0    {}
Name: params, dtype: object

### Run motion correction with the given parameter space

In [24]:
mes_df = load_mesmerize(fish)
run_mesmerize(mes_df)
mes_df = mes_df.caiman.reload_from_disk()

Batch exists. Loading batch.pickle
Running ca450d2a-f550-4abd-99ba-771d2e4dd7dd with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/Rapid/DOI/Bruker/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/mesmerize-batch/ca450d2a-f550-4abd-99ba-771d2e4dd7dd/ca450d2a-f550-4abd-99ba-771d2e4dd7dd-image_els__d1_1024_d2_1024_d3_1_order_F_frames_2046.mmap
finished computing correlation image


In [25]:
# See the rows that were successfully run
for output in mes_df.loc[:, 'outputs']:
    print(output['success'])

True


### Visualize outputs

In [28]:
# Compare the raw image to motion corrected ones
mes_df = load_mesmerize(fish)
mcorr_iw = visualize_mesmerize(mes_df, 'mcorr')
mcorr_iw.show()

Batch exists. Loading batch.pickle
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_9dpf_fed_glucose_25mM_23_20240111/mesmerize-batch/fdc1da29-fd3b-4902-85a1-8d7fdfe44eb4/fdc1da29-fd3b-4902-85a1-8d7fdfe44eb4-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap


RFBOutputContext()

In [29]:
mcorr_iw.plot.canvas.close()

In [26]:
# Visualize the difference of each frame from the mean
mcorr_diff_iw = visualize_diff(mes_df)
mcorr_diff_iw.show()

Decode mmap filename /Volumes/Rapid/bruker2P_data/neurod1rsChRmine_elavl3H2BGCaMP8m_8dpf_7_20231101/mesmerize-batch/147592e6-8946-4673-aae4-1916350d3fec/147592e6-8946-4673-aae4-1916350d3fec-image_els__d1_512_d2_512_d3_1_order_F_frames_2250.mmap


RFBOutputContext()

    64243017 [base_comm.py:            get_comm():212] [92926] No such comm: d19d4a46f0a64780ae8c9b1a81063406
    64243018 [base_comm.py:            get_comm():212] [92926] No such comm: d19d4a46f0a64780ae8c9b1a81063406


In [28]:
mcorr_diff_iw.plot.canvas.close()

### Choose an index with the best parameters

In [30]:
# Change the row index here that you want to keep
# If you want to delete all rows to start over, keep the list empty
indices = [0]

In [31]:
# Removes every row except for the chosen index in the indices list
df = load_mesmerize(fish)
mes_df = clean_mesmerize(df, indices)  
mes_df

Batch exists. Loading batch.pickle


,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,img_stack_0,img_stack_0/image.tif,"{'main': {'strides': (100, 100), 'overlaps': (50, 50), 'max_shifts': (30, 30), 'max_deviation_rigid': 10, 'pw_rigid'...",{'mean-projection-path': fdc1da29-fd3b-4902-85a1-8d7fdfe44eb4/fdc1da29-fd3b-4902-85a1-8d7fdfe44eb4_mean_projection.n...,2024-03-08T10:12:37,2024-03-08T10:13:33,52.77 sec,None,fdc1da29-fd3b-4902-85a1-8d7fdfe44eb4


## Run motion correction on all planes

In [32]:
# Adds every other plane in the volume with the same parameters as the leftover, chosen index
fish.process_mesmerize_filestructure()
mes_df = add_volume(fish, algo='mcorr')  
mes_df

Batch exists. Loading batch.pickle
Batch exists. Loading batch.pickle
Batch exists. Loading batch.pickle


,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,img_stack_0,img_stack_0/image.tif,"{'main': {'strides': (100, 100), 'overlaps': (50, 50), 'max_shifts': (30, 30), 'max_deviation_rigid': 10, 'pw_rigid'...",{'mean-projection-path': fdc1da29-fd3b-4902-85a1-8d7fdfe44eb4/fdc1da29-fd3b-4902-85a1-8d7fdfe44eb4_mean_projection.n...,2024-03-08T10:12:37,2024-03-08T10:13:33,52.77 sec,None,fdc1da29-fd3b-4902-85a1-8d7fdfe44eb4
1,mcorr,img_stack_1,img_stack_1/image.tif,"{'main': {'strides': (100, 100), 'overlaps': (50, 50), 'max_shifts': (30, 30), 'max_deviation_rigid': 10, 'pw_rigid'...",None,2024-03-08T10:14:08,None,None,None,f0ddec55-3b28-4304-ae72-cb0f0b409124
2,mcorr,img_stack_2,img_stack_2/image.tif,"{'main': {'strides': (100, 100), 'overlaps': (50, 50), 'max_shifts': (30, 30), 'max_deviation_rigid': 10, 'pw_rigid'...",None,2024-03-08T10:14:08,None,None,None,2bcb969e-f12c-410b-bf80-a121036c1537
3,mcorr,img_stack_3,img_stack_3/image.tif,"{'main': {'strides': (100, 100), 'overlaps': (50, 50), 'max_shifts': (30, 30), 'max_deviation_rigid': 10, 'pw_rigid'...",None,2024-03-08T10:14:08,None,None,None,3fef463a-ef1f-43d8-bb04-2d134c4c055e
4,mcorr,img_stack_4,img_stack_4/image.tif,"{'main': {'strides': (100, 100), 'overlaps': (50, 50), 'max_shifts': (30, 30), 'max_deviation_rigid': 10, 'pw_rigid'...",None,2024-03-08T10:14:08,None,None,None,1bfae04a-6f13-470e-9ac8-1e4505eb951f


In [33]:
# Run mesmerize on the entire volume
run_mesmerize(mes_df)
mes_df = mes_df.caiman.reload_from_disk()

# See the rows that were successfully run
for output in mes_df.loc[:, 'outputs']:
    print(output['success'])

Running f0ddec55-3b28-4304-ae72-cb0f0b409124 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_9dpf_fed_glucose_25mM_23_20240111/mesmerize-batch/f0ddec55-3b28-4304-ae72-cb0f0b409124/f0ddec55-3b28-4304-ae72-cb0f0b409124-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap
finished computing correlation image
Running 2bcb969e-f12c-410b-bf80-a121036c1537 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_9dpf_fed_glucose_25mM_23_20240111/mesmerize-batch/2bcb969e-f12c-410b-bf80-a121036c1537/2bcb969e-f12c-410b-bf80-a121036c1537-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap
finished computing correlation image
Running 3fef463a-ef1f-43d8-bb04-2d134c4c055e with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCa

### Visualize outputs

In [78]:
# Visualize the motion corrected output of all planes
mes_df = load_mesmerize(fish)
mcorr_iw = visualize_mesmerize(mes_df, 'mcorr')
mcorr_iw.show()

Batch exists. Loading batch.pickle
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_eggwater_0mM_18_20231206/mesmerize-batch/8615633a-d6e9-4b95-ac1c-12ffef3a0001/8615633a-d6e9-4b95-ac1c-12ffef3a0001-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_eggwater_0mM_18_20231206/mesmerize-batch/9e577156-6a00-4e85-a296-b0674bd27e85/9e577156-6a00-4e85-a296-b0674bd27e85-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_eggwater_0mM_18_20231206/mesmerize-batch/319c1a32-011b-4f2a-8f09-f6e89d1cc576/319c1a32-011b-4f2a-8f09-f6e89d1cc576-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_eggwater_0mM_18_20231206/mesmerize-batch/7f70d193-6785-4135-9fef-685b1387c603/7f70d193-6785-4135-9fef-685b1387c603-image_els__d1_512_d2_512_d3_1_order_

RFBOutputContext()

In [79]:
mcorr_iw.plot.canvas.close()